In [1]:
import sys
sys.path.append('../')

import collections
import os
import random
from pathlib import Path
import logging
import shutil
import time
from packaging import version
from collections import defaultdict

from tqdm import tqdm
import numpy as np
import gzip
import torch
import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import torch.backends.cudnn as cudnn

from src.param import parse_args
from src.utils import LossMeter
from src.dist_utils import reduce_dict
from transformers import T5Tokenizer, T5TokenizerFast
from src.tokenization import P5Tokenizer, P5TokenizerFast
from src.pretrain_model import P5Pretraining

_use_native_amp = False
_use_apex = False

# Check if Pytorch version >= 1.6 to switch between Native AMP and Apex
if version.parse(torch.__version__) < version.parse("1.6"):
    from transormers.file_utils import is_apex_available
    if is_apex_available():
        from apex import amp
    _use_apex = True
else:
    _use_native_amp = True
    from torch.cuda.amp import autocast

from src.trainer_base import TrainerBase

import pickle

def load_pickle(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)


def save_pickle(data, filename):
    with open(filename, "wb") as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
        
import json

def load_json(file_path):
    with open(file_path, "r") as f:
        return json.load(f)
    
def ReadLineFromFile(path):
    lines = []
    with open(path,'r') as fd:
        for line in fd:
            lines.append(line.rstrip('\n'))
    return lines

def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

In [2]:
class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self
        
args = DotDict()

args.distributed = False
args.multiGPU = True
args.fp16 = True
args.train = "yelp"
args.valid = "yelp"
args.test = "yelp"
args.batch_size = 16
args.optim = 'adamw' 
args.warmup_ratio = 0.02
args.lr = 1e-3
args.num_workers = 4
args.clip_grad_norm = 1.0
args.losses = 'rating,sequential,explanation,review,traditional'
args.backbone = 't5-small' # small or base
args.output = 'snap/yelp-small'
args.epoch = 10
args.local_rank = 0

args.comment = ''
args.train_topk = -1
args.valid_topk = -1
args.dropout = 0.1

args.tokenizer = 'p5'
args.max_text_length = 512
args.do_lower_case = False
args.word_mask_rate = 0.15
args.gen_max_length = 64

args.weight_decay = 0.01
args.adam_eps = 1e-6
args.gradient_accumulation_steps = 1

'''
Set seeds
'''
args.seed = 2022
torch.manual_seed(args.seed)
random.seed(args.seed)
np.random.seed(args.seed)

'''
Whole word embedding
'''
args.whole_word_embed = True

cudnn.benchmark = True
ngpus_per_node = torch.cuda.device_count()
args.world_size = ngpus_per_node

LOSSES_NAME = [f'{name}_loss' for name in args.losses.split(',')]
if args.local_rank in [0, -1]:
    print(LOSSES_NAME)
LOSSES_NAME.append('total_loss') # total loss

args.LOSSES_NAME = LOSSES_NAME

gpu = 1 #args.local_rank
args.gpu = gpu
args.rank = gpu
print(f'Process Launching at GPU {gpu}')

torch.cuda.set_device('cuda:{}'.format(gpu))

comments = []
dsets = []
if 'yelp' in args.train:
    dsets.append('yelp')
comments.append(''.join(dsets))
if args.backbone:
    comments.append(args.backbone)
comments.append(''.join(args.losses.split(',')))
if args.comment != '':
    comments.append(args.comment)
comment = '_'.join(comments)

from datetime import datetime
current_time = datetime.now().strftime('%b%d_%H-%M')

if args.local_rank in [0, -1]:
    run_name = f'{current_time}_GPU{args.world_size}'
    if len(comments) > 0:
        run_name += f'_{comment}'
    args.run_name = run_name
    print(args)

['rating_loss', 'sequential_loss', 'explanation_loss', 'review_loss', 'traditional_loss']
Process Launching at GPU 1
{'distributed': False, 'multiGPU': True, 'fp16': True, 'train': 'yelp', 'valid': 'yelp', 'test': 'yelp', 'batch_size': 16, 'optim': 'adamw', 'warmup_ratio': 0.02, 'lr': 0.001, 'num_workers': 4, 'clip_grad_norm': 1.0, 'losses': 'rating,sequential,explanation,review,traditional', 'backbone': 't5-small', 'output': 'snap/yelp-small', 'epoch': 10, 'local_rank': 0, 'comment': '', 'train_topk': -1, 'valid_topk': -1, 'dropout': 0.1, 'tokenizer': 'p5', 'max_text_length': 512, 'do_lower_case': False, 'word_mask_rate': 0.15, 'gen_max_length': 64, 'weight_decay': 0.01, 'adam_eps': 1e-06, 'gradient_accumulation_steps': 1, 'seed': 2022, 'whole_word_embed': True, 'world_size': 4, 'LOSSES_NAME': ['rating_loss', 'sequential_loss', 'explanation_loss', 'review_loss', 'traditional_loss', 'total_loss'], 'gpu': 1, 'rank': 1, 'run_name': 'Jul27_09-23_GPU4_yelp_t5-small_ratingsequentialexplanat

In [3]:
def create_config(args):
    from transformers import T5Config, BartConfig

    if 't5' in args.backbone:
        config_class = T5Config
    else:
        return None

    config = config_class.from_pretrained(args.backbone)
    config.dropout_rate = args.dropout
    config.dropout = args.dropout
    config.attention_dropout = args.dropout
    config.activation_dropout = args.dropout
    config.losses = args.losses

    return config


def create_tokenizer(args):
    from transformers import T5Tokenizer, T5TokenizerFast
    from src.tokenization import P5Tokenizer, P5TokenizerFast

    if 'p5' in args.tokenizer:
        tokenizer_class = P5Tokenizer

    tokenizer_name = args.backbone
    
    tokenizer = tokenizer_class.from_pretrained(
        tokenizer_name,
        max_length=args.max_text_length,
        do_lower_case=args.do_lower_case,
    )

    print(tokenizer_class, tokenizer_name)
    
    return tokenizer


def create_model(model_class, config=None):
    print(f'Building Model at GPU {args.gpu}')

    model_name = args.backbone

    model = model_class.from_pretrained(
        model_name,
        config=config
    )
    return model

In [4]:
config = create_config(args)

if args.tokenizer is None:
    args.tokenizer = args.backbone
    
tokenizer = create_tokenizer(args)

model_class = P5Pretraining
model = create_model(model_class, config)

model = model.cuda()

if 'p5' in args.tokenizer:
    model.resize_token_embeddings(tokenizer.vocab_size)
    
model.tokenizer = tokenizer

<class 'src.tokenization.P5Tokenizer'> t5-small
Building Model at GPU 1


Some weights of P5Pretraining were not initialized from the model checkpoint at t5-small and are newly initialized: ['encoder.whole_word_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Load Model

In [5]:
args.load = "../snap/yelp-small.pth"

# Load Checkpoint
from src.utils import load_state_dict, LossMeter, set_global_logging_level
from pprint import pprint

def load_checkpoint(ckpt_path):
    state_dict = load_state_dict(ckpt_path, 'cpu')
    results = model.load_state_dict(state_dict, strict=False)
    print('Model loaded from ', ckpt_path)
    pprint(results)

ckpt_path = args.load
load_checkpoint(ckpt_path)

from src.all_yelp_templates import all_tasks as task_templates

Model loaded from  ../snap/yelp-small-epoch10.pth
<All keys matched successfully>


#### Check Test Split

In [6]:
data_splits = load_pickle('../data/yelp/rating_splits_augmented.pkl')
test_review_data = data_splits['test']

In [7]:
len(test_review_data)

31636

In [8]:
test_review_data[0]

{'review_id': 'Vx-DPb4olxt1Zxf9dI8b2A',
 'useful': 0,
 'funny': 0,
 'cool': 0,
 'date': '2019-01-02 19:11:49',
 'unixReviewTime': '20190102191149',
 'reviewerID': 'Donht4mLJ4aO4FQhqHGJtw',
 'asin': '5eV8oUGdBXylwB7HeaDFOA',
 'overall': 4.0,
 'reviewText': 'Top notch cuisine and very friendly service. Everything here tastes very fresh. The rice pilaf is second to none!! Prices are very reasonable for the high quality food they serve!!',
 'explanation': 'Everything here tastes very fresh',
 'feature': 'taste'}

In [9]:
data_maps = load_json(os.path.join('../data', 'yelp', 'datamaps.json'))
print(len(data_maps['user2id'])) # number of users
print(len(data_maps['item2id'])) # number of items

30431
20033


### Test P5

In [10]:
from torch.utils.data import DataLoader, Dataset, Sampler
from src.pretrain_data import get_loader
from evaluate.utils import rouge_score, bleu_score, unique_sentence_percent, root_mean_square_error, mean_absolute_error, feature_detect, feature_matching_ratio, feature_coverage_ratio, feature_diversity
from evaluate.metrics4rec import evaluate_all

#### Evaluation - Rating

In [11]:
test_task_list = {'rating': ['1-10'] # or '1-6'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

gt_ratings = []
pred_ratings = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        gt_ratings.extend(batch['target_text'])
        pred_ratings.extend(results)
        
predicted_rating = [(float(r), float(p)) for (r, p) in zip(gt_ratings, pred_ratings) if p in [str(i/10.0) for i in list(range(10, 50))]]
RMSE = root_mean_square_error(predicted_rating, 5.0, 1.0)
print('RMSE {:7.4f}'.format(RMSE))
MAE = mean_absolute_error(predicted_rating, 5.0, 1.0)
print('MAE {:7.4f}'.format(MAE))

Data sources:  ['yelp']
compute_datum_info
1978


1978it [02:11, 14.99it/s]


RMSE  1.4685
MAE  1.0054


In [12]:
test_task_list = {'rating': ['1-6'] # or '1-10'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

gt_ratings = []
pred_ratings = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        gt_ratings.extend(batch['target_text'])
        pred_ratings.extend(results)
        
predicted_rating = [(float(r), float(p)) for (r, p) in zip(gt_ratings, pred_ratings) if p in [str(i/10.0) for i in list(range(10, 50))]]
RMSE = root_mean_square_error(predicted_rating, 5.0, 1.0)
print('RMSE {:7.4f}'.format(RMSE))
MAE = mean_absolute_error(predicted_rating, 5.0, 1.0)
print('MAE {:7.4f}'.format(MAE))

Data sources:  ['yelp']
compute_datum_info
1978


1978it [01:48, 18.18it/s]


RMSE  1.4868
MAE  1.0186


#### Evaluation - Sequential

In [13]:
test_task_list = {'sequential': ['2-13'] # or '2-3'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

all_info = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        beam_outputs = model.generate(
                batch['input_ids'].to('cuda'), 
                max_length=50, 
                num_beams=20,
                no_repeat_ngram_size=0, 
                num_return_sequences=20,
                early_stopping=True
        )
        generated_sents = model.tokenizer.batch_decode(beam_outputs, skip_special_tokens=True)
        for j, item in enumerate(zip(results, batch['target_text'], batch['source_text'])):
            new_info = {}
            new_info['target_item'] = item[1]
            new_info['gen_item_list'] = generated_sents[j*20: (j+1)*20]
            all_info.append(new_info)
            
gt = {}
ui_scores = {}
for i, info in enumerate(all_info):
    gt[i] = [int(info['target_item'])]
    pred_dict = {}
    for j in range(len(info['gen_item_list'])):
        try:
            pred_dict[int(info['gen_item_list'][j])] = -(j+1)
        except:
            pass
    ui_scores[i] = pred_dict
    
evaluate_all(ui_scores, gt, 5)
evaluate_all(ui_scores, gt, 10)

Data sources:  ['yelp']
compute_datum_info
1902


0it [00:00, ?it/s]/global/homes/z/zw241/.conda/envs/pt-1.10/lib/python3.9/site-packages/transformers/generation_utils.py:1632: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size
1902it [26:00,  1.22it/s]



NDCG@5	Rec@5	Hits@5	Prec@5	MAP@5	MRR@5
0.0403	0.0574	0.0574	0.0115	0.0346	0.0346

NDCG@10	Rec@10	Hits@10	Prec@10	MAP@10	MRR@10
0.0445	0.0703	0.0703	0.0070	0.0364	0.0364


('\nNDCG@10\tRec@10\tHits@10\tPrec@10\tMAP@10\tMRR@10\n0.0445\t0.0703\t0.0703\t0.0070\t0.0364\t0.0364',
 {'ndcg': 0.04450006114321309,
  'map': 0.03635002005578063,
  'recall': 0.07032302586178568,
  'precision': 0.007032302586178309,
  'mrr': 0.03635002005578063,
  'hit': 0.07032302586178568})

In [14]:
test_task_list = {'sequential': ['2-3'] # or '2-13'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

all_info = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        beam_outputs = model.generate(
                batch['input_ids'].to('cuda'), 
                max_length=50, 
                num_beams=20,
                no_repeat_ngram_size=0, 
                num_return_sequences=20,
                early_stopping=True
        )
        generated_sents = model.tokenizer.batch_decode(beam_outputs, skip_special_tokens=True)
        for j, item in enumerate(zip(results, batch['target_text'], batch['source_text'])):
            new_info = {}
            new_info['target_item'] = item[1]
            new_info['gen_item_list'] = generated_sents[j*20: (j+1)*20]
            all_info.append(new_info)
            
gt = {}
ui_scores = {}
for i, info in enumerate(all_info):
    gt[i] = [int(info['target_item'])]
    pred_dict = {}
    for j in range(len(info['gen_item_list'])):
        try:
            pred_dict[int(info['gen_item_list'][j])] = -(j+1)
        except:
            pass
    ui_scores[i] = pred_dict
    
evaluate_all(ui_scores, gt, 5)
evaluate_all(ui_scores, gt, 10)

Data sources:  ['yelp']
compute_datum_info
1902


1902it [25:20,  1.25it/s]



NDCG@5	Rec@5	Hits@5	Prec@5	MAP@5	MRR@5
0.0402	0.0568	0.0568	0.0114	0.0347	0.0347

NDCG@10	Rec@10	Hits@10	Prec@10	MAP@10	MRR@10
0.0447	0.0707	0.0707	0.0071	0.0365	0.0365


('\nNDCG@10\tRec@10\tHits@10\tPrec@10\tMAP@10\tMRR@10\n0.0447\t0.0707\t0.0707\t0.0071\t0.0365\t0.0365',
 {'ndcg': 0.044696482943397606,
  'map': 0.03653161745567516,
  'recall': 0.07065163813216785,
  'precision': 0.007065163813216525,
  'mrr': 0.03653161745567516,
  'hit': 0.07065163813216785})

#### Evaluation - Explanation

In [15]:
test_task_list = {'explanation': ['3-10'] # or '3-7' or '3-2'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

tokens_predict = []
tokens_test = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        tokens_predict.extend(results) 
        tokens_test.extend(batch['target_text'])
        
BLEU1 = bleu_score(tokens_test, tokens_predict, n_gram=1, smooth=False)
print('BLEU-1 {:7.4f}'.format(BLEU1))
BLEU4 = bleu_score(tokens_test, tokens_predict, n_gram=4, smooth=False)
print('BLEU-4 {:7.4f}'.format(BLEU4))

text_test = [' '.join(tokens) for tokens in tokens_test]
text_predict = [' '.join(tokens) for tokens in tokens_predict]
ROUGE = rouge_score(text_test, text_predict)  # a dictionary
for (k, v) in ROUGE.items():
    print('{} {:7.4f}'.format(k, v))

Data sources:  ['yelp']
compute_datum_info
1529


1529it [04:49,  5.27it/s]


BLEU-1 15.3002
BLEU-4  9.9617
rouge_1/f_score 70.6757
rouge_1/r_score 61.7185
rouge_1/p_score 86.1836
rouge_2/f_score 41.9679
rouge_2/r_score 33.5486
rouge_2/p_score 66.1482
rouge_l/f_score 34.6106
rouge_l/r_score 33.5335
rouge_l/p_score 73.9306


In [16]:
test_task_list = {'explanation': ['3-7'] # or '3-10' or '3-2'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

tokens_predict = []
tokens_test = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        tokens_predict.extend(results) 
        tokens_test.extend(batch['target_text'])
        
BLEU1 = bleu_score(tokens_test, tokens_predict, n_gram=1, smooth=False)
print('BLEU-1 {:7.4f}'.format(BLEU1))
BLEU4 = bleu_score(tokens_test, tokens_predict, n_gram=4, smooth=False)
print('BLEU-4 {:7.4f}'.format(BLEU4))

text_test = [' '.join(tokens) for tokens in tokens_test]
text_predict = [' '.join(tokens) for tokens in tokens_predict]
ROUGE = rouge_score(text_test, text_predict)  # a dictionary
for (k, v) in ROUGE.items():
    print('{} {:7.4f}'.format(k, v))

Data sources:  ['yelp']
compute_datum_info
1529


1529it [04:29,  5.68it/s]


BLEU-1 15.2346
BLEU-4  9.8952
rouge_1/f_score 70.4083
rouge_1/r_score 61.3351
rouge_1/p_score 86.1620
rouge_2/f_score 41.8598
rouge_2/r_score 33.3510
rouge_2/p_score 66.3752
rouge_l/f_score 34.5578
rouge_l/r_score 33.5047
rouge_l/p_score 74.1003


In [17]:
test_task_list = {'explanation': ['3-2'] # or '3-10' or '3-7'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

tokens_predict = []
tokens_test = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        tokens_predict.extend(results) 
        tokens_test.extend(batch['target_text'])
        
BLEU1 = bleu_score(tokens_test, tokens_predict, n_gram=1, smooth=False)
print('BLEU-1 {:7.4f}'.format(BLEU1))
BLEU4 = bleu_score(tokens_test, tokens_predict, n_gram=4, smooth=False)
print('BLEU-4 {:7.4f}'.format(BLEU4))

text_test = [' '.join(tokens) for tokens in tokens_test]
text_predict = [' '.join(tokens) for tokens in tokens_predict]
ROUGE = rouge_score(text_test, text_predict)  # a dictionary
for (k, v) in ROUGE.items():
    print('{} {:7.4f}'.format(k, v))

Data sources:  ['yelp']
compute_datum_info
1529


1529it [02:52,  8.87it/s]


BLEU-1  9.5202
BLEU-4  3.4515
rouge_1/f_score 63.8361
rouge_1/r_score 54.5817
rouge_1/p_score 79.9310
rouge_2/f_score 26.0032
rouge_2/r_score 19.7753
rouge_2/p_score 45.0442
rouge_l/f_score 26.9255
rouge_l/r_score 25.6947
rouge_l/p_score 64.2674


#### Evaluation - Review

In [18]:
test_task_list = {'review': ['4-3'] # or '4-2'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

gt_ratings = []
pred_ratings = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    if i > 50:
        break
    with torch.no_grad():
        results = model.generate_step(batch)
        gt_ratings.extend(batch['target_text'])
        pred_ratings.extend(results)
        
predicted_rating = [(float(r), round(float(p))) for (r, p) in zip(gt_ratings, pred_ratings)]
RMSE = root_mean_square_error(predicted_rating, 5.0, 1.0)
print('RMSE {:7.4f}'.format(RMSE))
MAE = mean_absolute_error(predicted_rating, 5.0, 1.0)
print('MAE {:7.4f}'.format(MAE))

Data sources:  ['yelp']
compute_datum_info
1978


51it [00:03, 13.66it/s]


RMSE  0.6350
MAE  0.3150


In [19]:
test_task_list = {'review': ['4-2'] # or '4-3'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

gt_ratings = []
pred_ratings = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    if i > 50:
        break
    with torch.no_grad():
        results = model.generate_step(batch)
        gt_ratings.extend(batch['target_text'])
        pred_ratings.extend(results)
        
predicted_rating = [(float(r), round(float(p))) for (r, p) in zip(gt_ratings, pred_ratings)]
RMSE = root_mean_square_error(predicted_rating, 5.0, 1.0)
print('RMSE {:7.4f}'.format(RMSE))
MAE = mean_absolute_error(predicted_rating, 5.0, 1.0)
print('MAE {:7.4f}'.format(MAE))

Data sources:  ['yelp']
compute_datum_info
1978


51it [00:03, 13.32it/s]


RMSE  0.6301
MAE  0.3113


#### Evaluation - Traditional

In [20]:
test_task_list = {'traditional': ['5-8']  # or '5-5'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

all_info = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        beam_outputs = model.generate(
                batch['input_ids'].to('cuda'), 
                max_length=50, 
                num_beams=20,
                no_repeat_ngram_size=0, 
                num_return_sequences=20,
                early_stopping=True
        )
        generated_sents = model.tokenizer.batch_decode(beam_outputs, skip_special_tokens=True)
        for j, item in enumerate(zip(results, batch['target_text'], batch['source_text'])):
            new_info = {}
            new_info['target_item'] = item[1]
            new_info['gen_item_list'] = generated_sents[j*20: (j+1)*20]
            all_info.append(new_info)
            
gt = {}
ui_scores = {}
for i, info in enumerate(all_info):
    gt[i] = [int(info['target_item'])]
    pred_dict = {}
    for j in range(len(info['gen_item_list'])):
        try:
            pred_dict[int(info['gen_item_list'][j])] = -(j+1)
        except:
            pass
    ui_scores[i] = pred_dict
    
evaluate_all(ui_scores, gt, 1)
evaluate_all(ui_scores, gt, 5)
evaluate_all(ui_scores, gt, 10)

Data sources:  ['yelp']
compute_datum_info
1902


1902it [27:03,  1.17it/s]



NDCG@1	Rec@1	Hits@1	Prec@1	MAP@1	MRR@1
0.0648	0.0648	0.0648	0.0648	0.0648	0.0648

NDCG@5	Rec@5	Hits@5	Prec@5	MAP@5	MRR@5
0.1203	0.1729	0.1729	0.0346	0.1030	0.1030

NDCG@10	Rec@10	Hits@10	Prec@10	MAP@10	MRR@10
0.1464	0.2543	0.2543	0.0254	0.1136	0.1136


('\nNDCG@10\tRec@10\tHits@10\tPrec@10\tMAP@10\tMRR@10\n0.1464\t0.2543\t0.2543\t0.0254\t0.1136\t0.1136',
 {'ndcg': 0.14637761309667505,
  'map': 0.11358604399335892,
  'recall': 0.25434589727580426,
  'precision': 0.02543458972758396,
  'mrr': 0.11358604399335892,
  'hit': 0.25434589727580426})

In [21]:
test_task_list = {'traditional': ['5-5']  # or '5-8'
}
test_sample_numbers = {'rating': 1, 'sequential': (1, 1, 1), 'explanation': 1, 'review': 1, 'traditional': (1, 1)}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

all_info = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        beam_outputs = model.generate(
                batch['input_ids'].to('cuda'), 
                max_length=50, 
                num_beams=20,
                no_repeat_ngram_size=0, 
                num_return_sequences=20,
                early_stopping=True
        )
        generated_sents = model.tokenizer.batch_decode(beam_outputs, skip_special_tokens=True)
        for j, item in enumerate(zip(results, batch['target_text'], batch['source_text'])):
            new_info = {}
            new_info['target_item'] = item[1]
            new_info['gen_item_list'] = generated_sents[j*20: (j+1)*20]
            all_info.append(new_info)
            
gt = {}
ui_scores = {}
for i, info in enumerate(all_info):
    gt[i] = [int(info['target_item'])]
    pred_dict = {}
    for j in range(len(info['gen_item_list'])):
        try:
            pred_dict[int(info['gen_item_list'][j])] = -(j+1)
        except:
            pass
    ui_scores[i] = pred_dict
    
evaluate_all(ui_scores, gt, 1)
evaluate_all(ui_scores, gt, 5)
evaluate_all(ui_scores, gt, 10)

Data sources:  ['yelp']
compute_datum_info
1902


1902it [26:32,  1.19it/s]



NDCG@1	Rec@1	Hits@1	Prec@1	MAP@1	MRR@1
0.0658	0.0658	0.0658	0.0658	0.0658	0.0658

NDCG@5	Rec@5	Hits@5	Prec@5	MAP@5	MRR@5
0.1223	0.1765	0.1765	0.0353	0.1046	0.1046

NDCG@10	Rec@10	Hits@10	Prec@10	MAP@10	MRR@10
0.1492	0.2603	0.2603	0.0260	0.1155	0.1155


('\nNDCG@10\tRec@10\tHits@10\tPrec@10\tMAP@10\tMRR@10\n0.1492\t0.2603\t0.2603\t0.0260\t0.1155\t0.1155',
 {'ndcg': 0.14922877417860922,
  'map': 0.11551674540320987,
  'recall': 0.26032664059675986,
  'precision': 0.026032664059679654,
  'mrr': 0.11551674540320987,
  'hit': 0.26032664059675986})